# 알약 위치 찾기

In [2]:
import os, cv2, math
import numpy as np

In [ ]:
# img 경로
path = './medinode_data_001_120/015/001.jpg'

img = cv2.imread(path)

if img is None:
    print('경로 확인 필요')
    
# 이미지 정보
img_w, img_h, img_c = img.shape

# 윤곽선 잡기
# canny
one = 50
two = 900
img_canny = cv2.Canny(img, one, two)

# # gaussian blur 표준편차 사용하여 이미지 균일하게 다룸
img_gaussian = cv2.GaussianBlur(img, ksize=(0,0), sigmaX=0.1)
img_gaussian_canny = cv2.Canny(img_gaussian, one, two)

# # 외곽선 검출 함수 cv2.findContours
img_count = cv2.findContours(img_gaussian_canny, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# img_count는 윤곽선의 정보를 담고있다.
# 기존 자료형이 튜플여서 수정불가. 그래서 리스트로 변환
ls_count = list(img_count[0])

if len(ls_count) < 3:
    raise('윤곽선 검출 실패!!! canny 쓰레스홀드 수정 필요')
else:
    drop_index = []
    index = 0
    img_count[0]
    for i in img_count[0]:
        point_x = []
        point_y = []    
        
        # 리스트 차원축소 안하면 for문 하나더 사용해야함.
        ls_2d = sum(i.tolist(), [])
        for x, y in ls_2d:
            point_x.append(x)
            point_y.append(y)
        
        # 윤곽선 나온 좌표에서 최대 최소 점의 위치 구하기
        max_x = max(point_x)
        min_x = min(point_x)
        max_y = max(point_y)
        min_y = min(point_y)   

        # 점과 점사이의 거리 이용해서 x, y 구하기
        len_x = abs(max_x - min_x)
        len_y = abs(max_y - min_y)

        # 현재는 알약 500 < 넓이 < 100000 면 알약이 아닌것으로 판단
        box_size = len_x * len_y
        if 500 < box_size < 100000:
            index += 1
        else:
            drop_index.append(index)
            index += 1

    drop_index.reverse()
    # 불필요한 객체 삭제
    for _ in drop_index:
        ls_count.pop(_)

# 박스 좌표 구하기
box_point = []
tag = 0
for ls in ls_count:
    p_x = []
    p_y = []
    ls_2d = sum(ls.tolist(), [])
    for x, y in ls_2d:
        p_x.append(x)
        p_y.append(y)
        
    # 윤곽선 나온 좌표에서 최대 최소 점의 위치 구하기
    max_x = max(p_x)
    min_x = min(p_x)
    max_y = max(p_y)
    min_y = min(p_y)

    # 박스의 각 좌표
    # 왼쪽위 x1, 오른쪽위 x2, 왼쪽아래 x3, 오른쪽아래 x4
    x1 = (min_x - 10, min_y - 10)
    x2 = (max_x + 10, min_y - 10)
    x3 = (min_x - 10, max_y + 10)
    x4 = (max_x + 10, max_y + 10)
    box_point.append((x1,x4))
    # 원본이미지에 알약 박스 그리기 - 색상 녹색
    cv2.rectangle(img, x1, x4, color=(0,255,0), thickness=2)

    # 각 알약 다른 창으로 시각화
    pill = img[min_y -1 : max_y + 10, min_x -10 : max_x + 10]
    window_name_hsv = f'pill {tag}'
    window_name_hsv_convert = f'hsv_convert {tag}'
    window_name_hsv_color = f'hsv_color {tag}'
        
    # 탐지된 알약 시각화
    cv2.imshow(window_name_hsv, pill)
    # cv2.imshow(window_name_hsv_convert, hsv_img)
    # cv2.imshow(window_name_hsv_color, hsv_show_white) # inrange 한 결과보기
    tag += 1

# 외곽선 검출 그리기 cv2.drawContours
draw = cv2.drawContours(img, contours=ls_count,contourIdx=-1, color=(0,0,255), thickness=2)

# cv2.imshow('draw', draw)

cv2.waitKey()
cv2.destroyAllWindows()

In [1]:
def location_pill(img_path):

    img = cv2.imread(path)

    if img is None:
        print('경로 확인 필요')
        
    # 이미지 정보
    img_w, img_h, img_c = img.shape

    # 윤곽선 잡기
    # canny
    one = 50
    two = 900
    img_canny = cv2.Canny(img, one, two)

    # # gaussian blur 표준편차 사용하여 이미지 균일하게 다룸
    img_gaussian = cv2.GaussianBlur(img, ksize=(0,0), sigmaX=0.1)
    img_gaussian_canny = cv2.Canny(img_gaussian, one, two)

    # # 외곽선 검출 함수 cv2.findContours
    img_count = cv2.findContours(img_gaussian_canny, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # img_count는 윤곽선의 정보를 담고있다.
    # 기존 자료형이 튜플여서 수정불가. 그래서 리스트로 변환
    ls_count = list(img_count[0])

    if len(ls_count) < 3:
        raise('윤곽선 검출 실패!!! canny 쓰레스홀드 수정 필요')
    else:
        drop_index = []
        index = 0
        img_count[0]
        for i in img_count[0]:
            point_x = []
            point_y = []    
            
            # 리스트 차원축소 안하면 for문 하나더 사용해야함.
            ls_2d = sum(i.tolist(), [])
            for x, y in ls_2d:
                point_x.append(x)
                point_y.append(y)
            
            # 윤곽선 나온 좌표에서 최대 최소 점의 위치 구하기
            max_x = max(point_x)
            min_x = min(point_x)
            max_y = max(point_y)
            min_y = min(point_y)   

            # 박스 중앙 좌표 구하기
            center_x = int(abs(max_x - min_x) / 2)
            center_y = int(abs(max_y - min_y) / 2)
            
            # 점과 점사이의 거리 이용해서 x, y 구하기
            len_x = abs(max_x - min_x)
            len_y = abs(max_y - min_y)

            # 현재는 알약 500 < 넓이 < 100000 면 알약이 아닌것으로 판단
            box_size = len_x * len_y
            if 500 < box_size < 100000:
                index += 1
            else:
                drop_index.append(index)
                index += 1

        drop_index.reverse()
        # 불필요한 객체 삭제
        for _ in drop_index:
            ls_count.pop(_)

    # 박스 좌표 구하기
    box_point = []
    tag = 0
    for ls in ls_count:
        p_x = []
        p_y = []
        ls_2d = sum(ls.tolist(), [])
        for x, y in ls_2d:
            p_x.append(x)
            p_y.append(y)
            
        # 윤곽선 나온 좌표에서 최대 최소 점의 위치 구하기
        max_x = max(p_x)
        min_x = min(p_x)
        max_y = max(p_y)
        min_y = min(p_y)

        # 박스의 각 좌표
        # 왼쪽위 x1, 오른쪽위 x2, 왼쪽아래 x3, 오른쪽아래 x4
        x1 = (min_x - 10, min_y - 10)
        x2 = (max_x + 10, min_y - 10)
        x3 = (min_x - 10, max_y + 10)
        x4 = (max_x + 10, max_y + 10)
        box_point.append((x1,x4))
        # 원본이미지에 알약 박스 그리기 - 색상 녹색
        cv2.rectangle(img, x1, x4, color=(0,255,0), thickness=2)

        # 각 알약 다른 창으로 시각화
        pill = img[min_y -1 : max_y + 10, min_x -10 : max_x + 10]
        window_name_hsv = f'pill {tag}'
        window_name_hsv_convert = f'hsv_convert {tag}'
        window_name_hsv_color = f'hsv_color {tag}'
            
        # 탐지된 알약 시각화
        # cv2.imshow(window_name_hsv, pill)
        # cv2.imshow(window_name_hsv_convert, hsv_img)
        # cv2.imshow(window_name_hsv_color, hsv_show_white) # inrange 한 결과보기
        tag += 1

    # 외곽선 검출 그리기 cv2.drawContours
    draw = cv2.drawContours(img, contours=ls_count,contourIdx=-1, color=(0,0,255), thickness=2)

    cv2.imshow('draw', draw)

    cv2.waitKey()
    cv2.destroyAllWindows()

In [2]:
import os, cv2, math
import numpy as np

img_path = './medinode_data_001_120/055/'
ls_img = os.listdir(img_path)

for _ in ls_img:
    path = img_path + _
    location_pill(path )